In [1]:
import sources.logins as lg
from sqlite3 import connect
import sources.parts_list_0802 as pl
import pandas as pd
from datetime import datetime as dt
import xlwings as xw

In [2]:
conn = connect('quotation.db')

In [3]:
sr = pl.summary_report()
task = sr.task_review(conn)

    Task  Receive  Repair  Ship
0    HCM        1       3     5
1  Hanoi        0       5    17


In [6]:

# new job no empty
q = f'''
        SELECT DISTINCT
                c.[rma no.],cu.customer_type,customer_name,cu.dealer,ac.territory,e.location,
                c.[create] as [Create Date],c.[Date Installed] AS [Installed Date],
                c.Model,c.Serial_no as Serial,sc.type,sc.mo,
                c.warranty_end_date as [Warranty End],c.recieve_date as [Received Date],
                c.in_inspect_date as [Inspection Date],c.[repair size] as [Size],c.[update user] as PIC,
                CASE
                    WHEN sc.status IS NULL THEN c.repair_status
                    ELSE sc.status || c.repair_status END AS [Status]
        
        FROM ((((consolidated c
             LEFT JOIN pf_code pf on c.[rma no.] = pf.[rma no.])
             LEFT JOIN customerS cu ON c.[customer_code] = cu.[No.])
             LEFT JOIN engineers e ON c.[update user] = e.exfm_code)
             LEFT JOIN scopes sc ON c.model = sc.model)
             LEFT JOIN acc_tbl_exp ac ON cu.[No.] = ac.[No.]
        WHERE c.[rma no.] > (select max(rma) from new_ml)
        
        ORDER BY c.[rma no.]
            
    '''
newjob = pd.read_sql(q,conn)
print(len(newjob))
newjob

8


,RMA No.,customer_type,CUSTOMER_NAME,dealer,Territory,location,Create Date,Installed Date,MODEL,Serial,Type,mo,Warranty End,Received Date,Inspection Date,Size,PIC,Status
0,FMSV2023100001,Private,Van Hanh Hospital,ETC,ETC,HCM,2023-10-02 00:00:00,2022-06-01 00:00:00,EC-530WL3,8C643K407,Colono,530,2023-06-01 00:00:00,2023-10-02 17:03:42,2023-10-04 13:05:17,Major,Nguyen,Inspection
1,FMSV2023100002,Private,Van Hanh Hospital,ETC,ETC,HCM,2023-10-02 00:00:00,2020-07-29 00:00:00,EC-250WL5,3C309K036,Colono,250,2021-07-29 00:00:00,2023-10-02 17:12:16,2023-10-04 12:38:24,Major,Nguyen,Inspection
2,FMSV2023100003,Private,Van Hanh Hospital,ETC,ETC,HCM,2023-10-02 00:00:00,2015-08-06 00:00:00,EC-250WL5,SC309A048,Colono,250,2016-08-05 00:00:00,2023-10-02 17:57:10,2023-10-04 12:50:43,Major,Nguyen,Inspection
3,FMSV2023100004,Private,Heci Center,ETC,ETC,HCM,2023-10-02 00:00:00,2018-08-13 00:00:00,EG-760Z,2G403K048,Gastro,760,2021-08-13 00:00:00,2023-10-02 19:17:13,2023-10-03 19:17:05,Minor,FFAP0112,Authorization
4,FMSV2023100005,None,Buon Ma Thuot Medical University Hospital,None,None,HCM,2023-10-03 00:00:00,2021-12-31 00:00:00,EC-720R/I,3C741K158,Colono,720,2022-12-31 00:00:00,2023-10-04 10:49:19,2023-10-04 10:54:06,Minor,Nguyen,Inspection
5,FMSV2023100006,Government,Hoa Binh General Hospital,TRANG THI,TRANG THI,Hanoi,2023-10-03 00:00:00,2021-10-01 00:00:00,EG-600WR,8G391K040,Gastro,600,2022-10-01 00:00:00,2023-10-03 18:41:32,2023-10-03 18:52:20,Major,thang,Inspection
6,FMSV2023100007,FF asset,"FUJIFILM Vietnam Co., Ltd.",None,Demo,HCM,2023-10-04 00:00:00,2016-02-24 00:00:00,EC-600WI,1C692K442,Colono,600,2017-02-24 00:00:00,2023-10-04 11:27:24,2023-10-04 11:43:12,Major,Nguyen,Inspection
7,FMSV2023100008,None,Nam Can Tho Medical Center,None,None,HCM,2023-10-04 00:00:00,2022-07-20 00:00:00,EG-530WR,9G361K611,Gastro,530,2023-07-20 00:00:00,2023-10-04 11:55:30,2023-10-04 12:31:25,Major,Nguyen,Inspection


In [5]:
# add missed rma by sn
# sn_checked = tuple(['4G361A481',
#                     '1C728K450',
#                     'RU039A058',
#                    '1C741K218',
#                    ])
q = f'''
        SELECT DISTINCT
                c.[rma no.],cu.customer_type,customer_name,cu.dealer,ac.territory,e.location,
                c.[create] as [Create Date],c.[Date Installed] AS [Installed Date],
                c.Model,c.Serial_no as Serial,sc.type,sc.mo,
                c.warranty_end_date as [Warranty End],c.recieve_date as [Received Date],
                c.in_inspect_date as [Inspection Date],c.[repair size] as [Size],c.[update user] as PIC,
                CASE
                    WHEN sc.status IS NULL THEN c.repair_status
                    ELSE sc.status || c.repair_status END AS [Status]
        
        FROM ((((consolidated c
             LEFT JOIN pf_code pf on c.[rma no.] = pf.[rma no.])
             LEFT JOIN customerS cu ON c.[customer_code] = cu.[No.])
             LEFT JOIN engineers e ON c.[update user] = e.exfm_code)
             LEFT JOIN scopes sc ON c.model = sc.model)
             LEFT JOIN acc_tbl_exp ac ON cu.[No.] = ac.[No.]
        WHERE c.serial_no IN {sn_checked}
        ORDER BY c.[rma no.]
            
    '''
newjob = pd.read_sql(q,conn)
print(len(newjob))
newjob

4


,RMA No.,customer_type,CUSTOMER_NAME,dealer,Territory,location,Create Date,Installed Date,MODEL,Serial,Type,mo,Warranty End,Received Date,Inspection Date,Size,PIC,Status
0,FMSV2023030040,Government,Viet Duc Hospital,TRANG THI,TRANG THI,HCM,2023-03-21 11:16:41,2017-12-11 00:00:00,EC-760R-V/I,1C728K450,Colono,760,2018-12-11 00:00:00,2023-03-21 13:17:05,2023-03-23 10:52:53,Major,thongle,Inspection
1,FMSV2023030041,Government,Hue University of Medicine and Pharmacy - Ginec,ETC,ETC,HCM,2023-03-21 13:27:22,2013-11-11 00:00:00,EG-530WR,4G361A481,Gastro,530,None,2023-03-23 14:16:53,2023-03-23 17:55:56,Major,thongle,Inspection
2,FMSV2023030069,Government,Thong Nhat Hospital,None,ETC,HCM,2023-03-27 10:18:52,2019-12-24 00:00:00,EC-720R/I,1C741K218,Colono,720,2020-12-24 00:00:00,2023-03-28 10:34:42,2023-03-28 11:15:38,Minor,thongle,Inspection
3,FMSV2023090265,Government,Hue University Of Medicine And Pharmacy - Ginec,ETC,ETC,HCM,2023-09-28 00:00:00,2013-11-11 00:00:00,EG-530UR2,RU039A058,Ultrasound,530,None,2023-09-28 18:29:54,2023-09-29 14:47:08,Major,Nguyen,Inspection


In [ ]:
pd.read_sql('SELECT [status] FROM new_ml',conn)

In [7]:
# add new job into ML

m_list = pd.read_sql('SELECT * FROM new_ml',conn)
bb = pl.summary_report()
cur = conn.cursor()
for i in range(len(newjob)):
    rma = newjob.iloc[i]['RMA No.']
    issue = bb.issues(conn,rma)
    i_no = i + len(m_list) + 1
    for j in range(len(newjob.iloc[i])):
        if newjob.iloc[i][j] == None:newjob.iloc[i][j] =''
            
    cur.execute(f"""INSERT INTO new_ml(rma,cus_type,Customer,Dealer,Location,installed_date,
                                       model,sn,scope_type,model2,wty_end_date,receive,inspection,repair_size,issue,
                                       pic,[status]) 
                    VALUES('{newjob.iloc[i]['RMA No.']}','{newjob.iloc[i]['customer_type']}',"{newjob.iloc[i]['CUSTOMER_NAME']}",
                    '{newjob.iloc[i]['dealer']}','{newjob.iloc[i]['location']}',
                    '{str(newjob.iloc[i]['Installed Date'])}','{newjob.iloc[i]['MODEL']}','{newjob.iloc[i]['Serial']}',
                    '{newjob.iloc[i]['Type']}','{newjob.iloc[i]['mo']}','{newjob.iloc[i]['Warranty End']}',
                    '{newjob.iloc[i]['Received Date']}','{str(newjob.iloc[i]['Inspection Date'])}','{newjob.iloc[i]['Size']}',
                    "{issue}","{newjob.iloc[i]['PIC']}","{newjob.iloc[i]['Status']}")
    """)

# save conn
conn.commit()
print('Done')

Done


In [9]:
q = '''select max([UPDATE_time]) AS max_update FROM new_ml'''
udt = pd.read_sql(q,conn)
update_time = udt['max_update'][0]
print(update_time)

# filter new update in consolidated 
cur = conn.cursor()
q = f'''

        SELECT c.[rma no.],c.customer_name,c.model,c.serial_no,
        
                c.recieve_date,c.[repair size],c.[update user],c.in_inspect_date,c.repair_status,c.[status info],
                c.[other id],c.last_update_time,c.approval,c.[repair user name],
                c.[update time]
        FROM consolidated c
        WHERE c.[update time] > '{update_time}'
        AND C.[rma no.] IN (SELECT rma FROM new_ml)
        
        ORDER BY 1
        
    '''
update_job = pd.read_sql(q,conn)
print(len(update_job))
# update_job

2023-09-30 08:46:01.092000
47


In [10]:
# update all
bb = pl.summary_report() # create issue
cur = conn.cursor()
m_list = pd.read_sql('SELECT * FROM new_ml',conn)
for i in range(len(update_job)):
# i = 0

    rma = update_job.iloc[i]['RMA No.']
    issue = bb.issues(conn,rma)
    print(issue)
    #replace none to empty
    for j in range(len(update_job.iloc[i])):
        if update_job.iloc[i][j] == None:update_job.iloc[i][j] =''

    # UPDATE GENERAL
    q = f""" UPDATE new_ml 
                SET receive = '{update_job.iloc[i]["RECIEVE_DATE"]}',
                inspection = '{update_job.iloc[i]['IN_INSPECT_DATE']}',
                exfm_status = '{update_job.iloc[i]['REPAIR_STATUS']}',
                approval = '{update_job.iloc[i]['Approval']}',
                repair_user = '{update_job.iloc[i]['Repair User Name']}',
                update_time = '{update_job.iloc[i]['Update Time']}',
                update_user = '{update_job.iloc[i]['Update User']}',
                exfm_info = '{update_job.iloc[i]['Status Info']}' 
                
                WHERE RMA = '{rma}'"""
    cur.execute(q)
    #update fixed item
    fixed_items = ['receive','report_id','inspection','repair_size']
    update_items = ['RECIEVE_DATE','Other ID','IN_INSPECT_DATE','Repair Size']
#     update_items[fixed_items.index('Report_ID')]       # compare each point
    
    for item in fixed_items:
        if str(m_list[m_list['rma']==rma][item])=='':
            cur.execute(f'''
                            UPDATE new_ml SET '{item}' = '{updatejob.iloc[i][update_items[fixed_items.index(item)]]}'
                            WHERE RMA = '{rma}'
                        ''')
            print(f'Update {rma}: {item} values {updatejob.iloc[i][update_items[fixed_items.index(item)]]}')
conn.commit()

BSA, FCT
CHA, FCT, BSA
VCA
CHA, BSA
FCT, CHA
CHA, DEC
CHA, FSB, PCB, BSA, FCT, DEC, LGB, VCA
BSA, FCT, VCA
RBS, FSA, BSA, FSB, FCT
BSA, FCT
FCT, VCA
CHA, BSA, LGB
CHA, FCT, BSA, FSA, DEC
OSA, CHA, BSA, LGB, DEC
CHA
ISA
BSA, FCT
BSA, FCT, FSB
CHA, BSA, FCT, DEC, FSA
FSA, CHA, VCA, FSB, FCT, DWA
BSA
FSB, DWA
LGB, BSA, VCAC
IRS
ISA
BSA, DRY, FSB
DRY, OSA
DWA, BSA
ISA
BSA, FCT, FSB, ISA, WGB
CHA, BSA, FSA, LPA
AWB, DRY, BSA
CHA, LGB, FCT, PCB
PCB
FSB, ANGL
CHA, FCT, BSA, LC, FSB, VCA
CHA, BSA, FCT, FSA
CHA, BSA, FCT, VCA, FSB
FSB, LGB, FSA
CHA, BSA, DWA
BSA, DEC
CHA, FSA
NOZ
ANGL, OSA
FSA
ISA, FSA, ANGL
CHA, FCT


In [11]:
# update empty status
bb = pl.summary_report() # create issue
cur = conn.cursor()
m_list = pd.read_sql('SELECT * FROM new_ml',conn)

q = f'''
        SELECT m.rma,m.customer,m.model,m.sn,C.[repair_status],c.[status info],m.exfm_status,m.exfm_info,
        c.approval,c.[update time],c.[update user],c.[repair user name],c.recieve_date,c.in_inspect_date,
        c.[repair size]
        
        FROM new_ml m
        LEFT JOIN consolidated c ON m.rma = c.[rma no.]
        
    '''
stt_info = pd.read_sql(q,conn)
display(stt_info)

for i in range(len(stt_info)):
# i = 0
#     print(i)
    rma = m_list.iloc[i]['rma']
    issue = bb.issues(conn,rma)
#     print(rma,issue)
    
    #replace none to empty
    for j in range(len(stt_info.iloc[i])):
        if stt_info.iloc[i][j] == None:stt_info.iloc[i][j] =''

    # UPDATE GENERAL
    cur.execute(f"""UPDATE new_ml SET issue = '{issue}',
    exfm_status = '{stt_info.iloc[i]['REPAIR_STATUS']}',
    exfm_info = '{stt_info.iloc[i]['Status Info']}',
    approval = '{stt_info.iloc[i]['Approval']}',
    update_time = '{stt_info.iloc[i]['Update Time']}',
    update_user = '{stt_info.iloc[i]['Update User']}',
    receive = '{stt_info.iloc[i]['RECIEVE_DATE']}',
    inspection = '{stt_info.iloc[i]['IN_INSPECT_DATE']}',
    repair_size = '{stt_info.iloc[i]['Repair Size']}'
    
    WHERE rma = '{rma}'""")
    
conn.commit()
print('Done')

,rma,customer,model,sn,REPAIR_STATUS,Status Info,exfm_status,exfm_info,Approval,Update Time,Update User,Repair User Name,RECIEVE_DATE,IN_INSPECT_DATE,Repair Size
0,FMSV2023020019,"FUJIFILM Vietnam Co., Ltd.",EC-760ZP-V/M,2C730K027,Authorization,Awating Contract billing,Authorization,Awating Contract billing,Approval,2023-10-03 16:50:07.970000,FFAP0062,None,2023-02-08 18:31:56,2023-02-13 17:33:19,Major
1,FMSV2023020022,FTYO - Loaner Asset,EC-760ZP-V/M,2C730K032,Inspection,PO Received,Inspection,PO Received,None,2023-08-14 12:49:53.215000,thongle,None,2023-02-08 18:54:02,2023-02-14 12:09:53,Major
2,FMSV2023030060,Hoang Long Clinic,VP-7000,2V644K028,Inspection,Parts on Order,Inspection,Parts on Order,None,2023-08-09 11:49:51.353000,thang,None,2023-03-23 19:41:03,2023-03-31 12:26:02,Other
3,FMSV2023030075,"FUJIFILM Vietnam Co., Ltd.",EC-530WI,1C603K041,Authorization,None,Authorization,,Approval,2023-10-03 15:50:20.760000,FFAP0070,None,2023-03-30 10:56:51,2023-04-04 18:33:19,Major
4,FMSV2023040004,FTYO - Loaner Asset,EN-580T,1C675K021,Inspection,PO Received,Inspection,PO Received,None,2023-08-14 12:50:45.054000,thongle,None,2023-04-03 17:13:24,2023-04-11 13:22:41,Major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,FMSV2023100004,Heci Center,EG-760Z,2G403K048,Authorization,None,Authorization,,Approval,2023-10-03 19:20:34.787000,FFAP0112,None,2023-10-02 19:17:13,2023-10-03 19:17:05,Minor
317,FMSV2023100005,Buon Ma Thuot Medical University Hospital,EC-720R/I,3C741K158,Inspection,Waiting for Next Process Available,Inspection,Waiting for Next Process Available,None,2023-10-04 18:26:38.827000,Nguyen,None,2023-10-04 10:49:19,2023-10-04 10:54:06,Minor
318,FMSV2023100006,Hoa Binh General Hospital,EG-600WR,8G391K040,Inspection,Awating Parts billing,Inspection,Awating Parts billing,None,2023-10-03 19:18:01.732000,thang,None,2023-10-03 18:41:32,2023-10-03 18:52:20,Major
319,FMSV2023100007,"FUJIFILM Vietnam Co., Ltd.",EC-600WI,1C692K442,Inspection,Awating Internal Approval,Inspection,Awating Internal Approval,None,2023-10-04 13:09:42.046000,Nguyen,None,2023-10-04 11:27:24,2023-10-04 11:43:12,Major


Done


In [12]:
#check no receive and inspection
q = '''
        SELECT rma,receive,inspection,
        c.[recieve_date],c.[in_inspect_date],m.issue,m.repair_size,c.[repair size]
        
        FROM new_ml m
        LEFT JOIN consolidated c ON m.rma = c.[rma no.]
        WHERE (receive IS NULL OR inspection IS NULL)
        
    '''
receive_inspect = pd.read_sql(q,conn)
receive_inspect

,rma,receive,inspection,RECIEVE_DATE,IN_INSPECT_DATE,issue,repair_size,Repair Size


In [ ]:
# add receive inspect infomation
# receive_inspect
for i in range(len(receive_inspect)):
# i = 0
#     print(i)
    rma = receive_inspect.iloc[i]['rma']
    issue = bb.issues(conn,rma)
#     print(rma,issue)
    
    #replace none to empty
    for j in range(len(receive_inspect.iloc[i])):
        if receive_inspect.iloc[i][j] == None:receive_inspect.iloc[i][j] =''

    # UPDATE GENERAL
    cur.execute(f"""UPDATE new_ml SET issue = '{issue}',
    
    receive = '{receive_inspect.iloc[i]['RECIEVE_DATE']}',
    inspection = '{receive_inspect.iloc[i]['IN_INSPECT_DATE']}',
    repair_size = '{receive_inspect.iloc[i]['Repair Size']}'
    
    WHERE rma = '{rma}'""")
    
conn.commit()
print('Done')

In [ ]:
#filter completed
q = '''
        SELECT  rma,customer,model, sn,repair_size,issue,pic,note,location,
            receive,quotation,confirmation,part_order_date, part_received_date,
            start_repair,qc_date,return_date 
        FROM new_ml 
        WHERE status LIKE '%Completed repair' 
        ORDER BY rma
'''
completed_list = pd.read_sql(q,conn)
completed_list

In [ ]:
#delete job completed from new_ml
cur.execute('DELETE FROM new_ml WHERE status LIKE "%Completed repair"')
conn.commit()

In [ ]:
q = '''
        SELECT * FROM new_ml
        WHERE exfm_status IS NULL
        
    '''
pd.read_sql(q,conn)

In [13]:
# 
q = "SELECT * FROM new_ml where exfm_status is null"
pd.read_sql(q,conn)

,No,rma,cus_type,customer,dealer,location,installed_date,model,sn,scope_type,...,note,exfm_status,exfm_info,approval,repair_user,update_time,update_user,TAT_receive,TAT_PO,TAT_PART


#####  Export


In [14]:
file_name = lg.file_select(folder_name = 'files', end_with = '.xlsm')
xw.Book(file_name)


______________________________________________________________________
|  No.|  File Name                               |  Modified Time    |
|____________________________________________________________________|
|  1  |  New ML_231004_200825.xlsm               |  04-Oct-23 20:16  |
|____________________________________________________________________|
Select file (Default 1): 1
"New ML_231004_200825.xlsm" is selected


<Book [New ML_231004_200825.xlsm]>

In [15]:
# final
q = '''SELECT * FROM new_ml'''
m_list_final = pd.read_sql(q,conn)
for i in range(len(m_list_final)):
    rma = m_list_final.iloc[i]['rma']
    issue = bb.issues(conn,rma)
    
    #replace none to empty
    for j in range(len(m_list_final.iloc[i])):
#         if m_list_final.iloc[i][j] == None: m_list_final.iloc[i][j] =''
        if m_list_final.iloc[i][j] == 'None': m_list_final.iloc[i][j] =''
           
m_list_final = m_list_final.drop(['TAT_receive','TAT_PO','TAT_PART'],axis=1)
dtmp = dt.now().strftime('%y%m%d')
file_name = f'ML_{dtmp}.xlsx'
m_list_final.to_excel(file_name,index = False)
xw.Book(file_name)

<Book [ML_231005.xlsx]>

In [ ]:
# copy file from files folder to new ML
file = lg.file_select(folder_name='files',start_with='New ML')
# to destination folder

try:
    copy_selected_file(des_folder='New ML',file_name = file)
    print(f'Copy sucess file {file}')
except Exception as e:
    print(e)


In [ ]:
# delete Master list in folder file
import os
import pathlib
path = pathlib.Path().absolute()
folder_name = 'files'
file_name = file.split('\\')[-1]
for file in os.listdir(folder_name):
    
    if 'New ML' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,folder_name,file))
        
# delete file ML
for file in os.listdir():
    
    if 'ML_' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,file))

for file in os.listdir('New ML'):
    
    if 'ML_' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,file))

In [ ]:
for file in os.listdir('New ML'):
    
    if 'ML_' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,file))

In [ ]:
# compare new ML and ExFM
q = '''
        SELECT [rma no.],customer_name, model, serial_no,repair_status,
        [update user],[update time],[status info]
        
        FROM consolidated
        WHERE [rma no.] NOT IN (SELECT rma FROM new_ml)
        
'''
gap = pd.read_sql(q,conn)
gap.to_excel('exfm_gap_230816.xlsx',index = False)

In [ ]:
q = f'''
        SELECT distinct rma,customer,nm.dealer,cs.dealer,c.customer_code
        
        FROM (new_ml nm
        LEFT JOIN consolidated c ON nm.rma = c.[rma no.])
        LEFT JOIN customers cs ON c.customer_code = cs.[no.]
    '''
d =pd.read_sql(q,conn)
d.to_excel('inspection/dealer.xlsx',index = False)

In [ ]:
q = 'SELECT DEALER from customers WHERE [NO.] = "FMSV00100"'
pd.read_sql(q,conn)

In [ ]:
conn.close()
conn = connect('quotation.db')

In [ ]:
conn.close()